RDD:

Transformations:
    map(f:T->U)                                :RDD[T]->RDD[U]
    filter(f:T->Bool)                          :RDD[T]->RDD[T]
    flatMap(f:T->Seq[U])                       :RDD[T]->RDD[U]
    sample(Withreplacemen:Bool,fration:Float)  :RDD[T]->RDD[T]                 (sampling)
    groupByKey()                               :RDD[(K,V)]->RDD[(K,Seq[V])]    (same key will be group together)
    reduceByKey(f:(V,V)->V))                   :RDD[(K,V)]->RDD[(K,V)]
    union()                                    :(RDD[T],RDD[T])->RDD[T]
    join()                                     :(RDD[(K,V)],RDD[(K,V)])->RDD[(K,(V,W))]
    cogroup()                                  :(RDD[(K,V)],RDD[(K,W)])->RDD[(K,(Seq(V),Seq(W))]
    crossProduct()                             :(RDD[T],RDD[U])->RDD[(T,U)]
    mapValues(f:V->W)                          :RDD[(K,V)]->RDD[(K,W)]          (Preserves partitioning) 
    sortBy(f:V)                                :RDD[(K,V)]->RDD[(K,V)]
    sortBysortBykey()                          :RDD[(K,V)]->RDD[(K,V)] 
    partitionBy(p:Partitioner[K])              :RDD[(K,V)]->RDD[(K,V)]
    mapPartitions(f)                           :Do actions inside each partitions
    glom()                                     :show the elements inside each partitions
    mapPartitionsWithIndex(f:index,list->somrthing):
    
   
Actions:
    count()                                    :RDD[T]->Long
    collect()                                  :RDD[T]->Seq[T]
    reduce(f:(T,T)->T)                         :RDD[T]->T          
    lookup(k:K)                                :RDD[(K,V)]->Seq[V] (on hash/range partitioned RDDs)    
    save(path:String)                          :outputs RDD to a storage system, e.g.,HDFS

In [148]:
#read textFile with 4 partitions
fruits = sc.textFile('fruits.txt',4)
print(fruits.collect())
#cache()
fruits.cache()
fruits.unpersist()
print(fruits.first())
print(fruits.take(3))

['apple', 'banana', 'banana', 'canary melon', 'grap', 'lemon', 'orange', 'pineapple', 'strawberry', 'strawberry']
apple
['apple', 'banana', 'banana']


In [149]:
A = sc.parallelize(range(1, 10))    #creat a rdd
A.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [150]:
#map
B = A.map(lambda x: (x,x+1))
print(B.collect())
#mapValues
print(B.mapValues(lambda x:x+10).collect())
#flatMap
C = B.flatMap(lambda x: list(x))
print(C.collect())

[(1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10)]
[(1, 12), (2, 13), (3, 14), (4, 15), (5, 16), (6, 17), (7, 18), (8, 19), (9, 20)]
[1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10]


In [151]:
#filter
t = 5
A.filter(lambda x: x < t).collect()

[1, 2, 3, 4]

In [152]:
#sample
C.sample(False,0.3).collect()

[5, 6, 8]

In [153]:
#groupByKey
B = A.map(lambda x: (1,x) if x>5 else (2,x))
print(B.collect())
print(B.groupByKey().count())
#reduceByKey
print(B.reduceByKey(lambda a,b:a if a>b else b).collect())

[(2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (1, 6), (1, 7), (1, 8), (1, 9)]
2
[(1, 9), (2, 5)]


In [154]:
#union() 
print(A.collect())
print(C.collect())
print(A.union(C).collect())
#join() 
print('join')
print(B.collect())
D = A.map(lambda x: (1,x) if x<5 else (2,x))
print(D.collect())
print(B.join(D).collect())

[1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10]
join
[(2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (1, 6), (1, 7), (1, 8), (1, 9)]
[(1, 1), (1, 2), (1, 3), (1, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9)]
[(1, (6, 1)), (1, (6, 2)), (1, (6, 3)), (1, (6, 4)), (1, (7, 1)), (1, (7, 2)), (1, (7, 3)), (1, (7, 4)), (1, (8, 1)), (1, (8, 2)), (1, (8, 3)), (1, (8, 4)), (1, (9, 1)), (1, (9, 2)), (1, (9, 3)), (1, (9, 4)), (2, (1, 5)), (2, (1, 6)), (2, (1, 7)), (2, (1, 8)), (2, (1, 9)), (2, (2, 5)), (2, (2, 6)), (2, (2, 7)), (2, (2, 8)), (2, (2, 9)), (2, (3, 5)), (2, (3, 6)), (2, (3, 7)), (2, (3, 8)), (2, (3, 9)), (2, (4, 5)), (2, (4, 6)), (2, (4, 7)), (2, (4, 8)), (2, (4, 9)), (2, (5, 5)), (2, (5, 6)), (2, (5, 7)), (2, (5, 8)), (2, (5, 9))]


In [155]:
#sortBysortBy()
print(A.sortBy(lambda x:x).collect())
#sortBysortBykey()
print(B.sortByKey().collect())

[1, 2, 3, 4, 5, 6, 7, 8, 9]
[(1, 6), (1, 7), (1, 8), (1, 9), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5)]


SQL:
SQL:
SQL:
SQL:
SQL:
SQL:
SQL:
More detail please go to the notebook(sparksql.ipynb) which Dr.Yi provided.

In [44]:
#set up
from pyspark.sql.functions import *
df = spark.read.csv('building.csv', header=True, inferSchema=True)

In [45]:
# show the content of the dataframe
df.show()

+----------+-----------+-----------+-----------+------------+
|BuildingID|BuildingMgr|BuildingAge|HVACproduct|     Country|
+----------+-----------+-----------+-----------+------------+
|         1|         M1|         25|     AC1000|         USA|
|         2|         M2|         27|     FN39TG|      France|
|         3|         M3|         28|     JDNS77|      Brazil|
|         4|         M4|         17|     GG1919|     Finland|
|         5|         M5|          3|    ACMAX22|   Hong Kong|
|         6|         M6|          9|     AC1000|   Singapore|
|         7|         M7|         13|     FN39TG|South Africa|
|         8|         M8|         25|     JDNS77|   Australia|
|         9|         M9|         11|     GG1919|      Mexico|
|        10|        M10|         23|    ACMAX22|       China|
|        11|        M11|         14|     AC1000|     Belgium|
|        12|        M12|         26|     FN39TG|     Finland|
|        13|        M13|         25|     JDNS77|Saudi Arabia|
|       

In [46]:
# Print the dataframe schema in a tree format
df.printSchema()

root
 |-- BuildingID: integer (nullable = true)
 |-- BuildingMgr: string (nullable = true)
 |-- BuildingAge: integer (nullable = true)
 |-- HVACproduct: string (nullable = true)
 |-- Country: string (nullable = true)



In [47]:
# Create an RDD from the dataframe
dfrdd = df.rdd
dfrdd.take(3)

[Row(BuildingID=1, BuildingMgr='M1', BuildingAge=25, HVACproduct='AC1000', Country='USA'),
 Row(BuildingID=2, BuildingMgr='M2', BuildingAge=27, HVACproduct='FN39TG', Country='France'),
 Row(BuildingID=3, BuildingMgr='M3', BuildingAge=28, HVACproduct='JDNS77', Country='Brazil')]

In [48]:
# Retrieve specific columns from the dataframe
df.select('BuildingID', 'Country').show()

+----------+------------+
|BuildingID|     Country|
+----------+------------+
|         1|         USA|
|         2|      France|
|         3|      Brazil|
|         4|     Finland|
|         5|   Hong Kong|
|         6|   Singapore|
|         7|South Africa|
|         8|   Australia|
|         9|      Mexico|
|        10|       China|
|        11|     Belgium|
|        12|     Finland|
|        13|Saudi Arabia|
|        14|     Germany|
|        15|      Israel|
|        16|      Turkey|
|        17|       Egypt|
|        18|   Indonesia|
|        19|      Canada|
|        20|   Argentina|
+----------+------------+



In [49]:
df1 = df.where("Country<'USA'").select('BuildingID', lit(df['BuildingID']*2).alias('new colum'))
#where can be replace by filter                create new column and give a name
df1.show()

+----------+---------+
|BuildingID|new colum|
+----------+---------+
|         2|        4|
|         3|        6|
|         4|        8|
|         5|       10|
|         6|       12|
|         7|       14|
|         8|       16|
|         9|       18|
|        10|       20|
|        11|       22|
|        12|       24|
|        13|       26|
|        14|       28|
|        15|       30|
|        16|       32|
|        17|       34|
|        18|       36|
|        19|       38|
|        20|       40|
+----------+---------+



In [174]:
#orderBy()
df.where("BuildingID<='3'").orderBy('BuildingAge',ascending=False).show()

+----------+-----------+-----------+-----------+-------+
|BuildingID|BuildingMgr|BuildingAge|HVACproduct|Country|
+----------+-----------+-----------+-----------+-------+
|         3|         M3|         28|     JDNS77| Brazil|
|         2|         M2|         27|     FN39TG| France|
|         1|         M1|         25|     AC1000|    USA|
+----------+-----------+-----------+-----------+-------+



In [175]:
# Use GroupBy clause with dataframe 
df.groupBy('HVACProduct').count().show(truncate=False)

+-----------+-----+
|HVACProduct|count|
+-----------+-----+
|ACMAX22    |4    |
|AC1000     |4    |
|JDNS77     |4    |
|FN39TG     |4    |
|GG1919     |4    |
+-----------+-----+



In [179]:
#join and rename
df.show()
df1.show()
df.join(df1,'BuildingID').withColumnRenamed('new colum', 'my').show()

+----------+-----------+-----------+-----------+------------+
|BuildingID|BuildingMgr|BuildingAge|HVACproduct|     Country|
+----------+-----------+-----------+-----------+------------+
|         1|         M1|         25|     AC1000|         USA|
|         2|         M2|         27|     FN39TG|      France|
|         3|         M3|         28|     JDNS77|      Brazil|
|         4|         M4|         17|     GG1919|     Finland|
|         5|         M5|          3|    ACMAX22|   Hong Kong|
|         6|         M6|          9|     AC1000|   Singapore|
|         7|         M7|         13|     FN39TG|South Africa|
|         8|         M8|         25|     JDNS77|   Australia|
|         9|         M9|         11|     GG1919|      Mexico|
|        10|        M10|         23|    ACMAX22|       China|
|        11|        M11|         14|     AC1000|     Belgium|
|        12|        M12|         26|     FN39TG|     Finland|
|        13|        M13|         25|     JDNS77|Saudi Arabia|
|       

In [52]:
#past exam example
dfTransaction = spark.read.csv('Transaction.csv', header=True, inferSchema=True)
dfProduct = spark.read.csv('Product2020.csv', header=True, inferSchema=True)

#The following two values will change during the grading.  
#For test purposes during the exam, you can set them to be
RequiredBrand= "Brand#23"
RequiredContainer = "MED BOX"

# s = dfTransaction.groupBy('Product_ID').sum('SaleQuantity')
# c = dfTransaction.groupBy('Product_ID').count()
# print(c.show())
select_condition = 'Brand = \'' + RequiredBrand + '\' and Container = \'' + RequiredContainer + '\''
dfProductSelect = dfProduct.where(select_condition).select("Product_ID").withColumnRenamed("Product_ID", "p_ID")
dfJoin = dfProductSelect.join(dfTransaction, dfProductSelect.p_ID == dfTransaction.Product_ID)
dfAvg = dfJoin.groupBy("p_ID").avg("SaleQuantity").withColumnRenamed("avg(SaleQuantity)", "avgQuantity")
dfFinal = dfAvg.join(dfJoin, dfAvg.p_ID == dfJoin.p_ID).filter("avgQuantity > SaleQuantity")
dfFinal.select('*', lit(dfFinal.SalePrice*dfFinal.SaleQuantity).alias('revenue')).groupby().sum('revenue').collect()

[Row(sum(revenue)=1016198410.83)]

Internal
Internal
Internal
Internal
Internal
Internal
Internal

In [180]:
#partitionBy()
#mapPartitions()
#glom()
#internal

def partitionsize(it): 
    yield len(list(it))

print(B.partitionBy(2,lambda x:x%2).mapPartitions(partitionsize).collect())  #size of each patition
print(B.partitionBy(2,lambda x:x%2).glom().collect())        #element of each partition
print(B.glom().collect())

[5, 4]
[[(2, 1), (2, 2), (2, 3), (2, 4), (2, 5)], [(1, 6), (1, 7), (1, 8), (1, 9)]]
[[(2, 1), (2, 2)], [(2, 3), (2, 4)], [(2, 5), (1, 6)], [(1, 7), (1, 8), (1, 9)]]


Algorithm
Algorithm
Algorithm
Algorithm
Algorithm
Algorithm
Algorithm
Algorithm

In [184]:
#used mapPartitions() 
x = 'abcccbcbcacaccacaabb'
y = 'abcccbcccacaccacaabb'
numPartitions = 4
rdd = sc.parallelize(zip(x,y), numPartitions)
print(rdd.glom().collect())

def f(i):
    for x in i:
        if x[0] > x[1]:
            yield '>'
            break
        elif x[0] < x[1]:
            yield '<'
            break

results = rdd.mapPartitions(f).collect()
print(results)
if len(results) == 0:
    print('=')
else:
    print(results[0])
    

[[('a', 'a'), ('b', 'b'), ('c', 'c'), ('c', 'c'), ('c', 'c')], [('b', 'b'), ('c', 'c'), ('b', 'c'), ('c', 'c'), ('a', 'a')], [('c', 'c'), ('a', 'a'), ('c', 'c'), ('c', 'c'), ('a', 'a')], [('c', 'c'), ('a', 'a'), ('a', 'a'), ('b', 'b'), ('b', 'b')]]
['<']
<


Graph:
Graph:
Graph:
Graph:
Graph:
Graph:
Graph:
Graph:
Graph:
Graph:
./hadoop/spark-3.0.3/bin/pyspark --packages graphframes:graphframes:0.8.1-spark3.0-s_2.12

In [11]:
sc.addPyFile("../graphframes-0.8.1-spark3.0-s_2.12.jar")

In [12]:
# Vertics DataFrame
v = spark.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 37),
  ("d", "David", 29),
  ("e", "Esther", 32),
  ("f", "Fanny", 38),
  ("g", "Gabby", 60)
], ["id", "name", "age"])

# Edges DataFrame
e = spark.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
  ("f", "c", "follow"),
  ("e", "f", "follow"),
  ("e", "d", "friend"),
  ("d", "a", "friend"),
  ("a", "e", "friend"),
  ("g", "e", "follow")
], ["src", "dst", "relationship"])

# Create a GraphFrame
g = GraphFrame(v, e)

g.vertices.show()
g.edges.show()

NameError: name 'GraphFrame' is not defined

Stream:
Stream:
Stream:
Stream:
Stream:
Stream:
Stream:
Stream:
Stream:
updateStateByKey(updateFunc) :keep the last time values, and the current input will be a list of somthing

In [57]:
#Q5
from pyspark.streaming import StreamingContext

ssc = StreamingContext(sc, 5)
# Provide a checkpointing directory. Required for stateful transformations
ssc.checkpoint("checkpoint")

numPartitions = 8
rdd = sc.textFile('adj_noun_pairs.txt', numPartitions)
rddQueue = rdd.randomSplit([1]*10, 123)


lines = ssc.queueStream(rddQueue)

def updateFunc(newValues, running):
    if running == None and len(newValues) != 0:
        return newValues[0]
    elif running != None and len(newValues) == 0:
        return running
    elif running != None and len(newValues) != 0:
        if len(newValues[0]) > len(running):
            return newValues[0]
        else:
            return running

def printre(rdd):
    print(rdd.take(5))
    
def printgood(rdd):
    print("GOOD:",rdd.map(lambda rdd: rdd[1]).take(1))
    print()

pairs  = lines.flatMap(lambda line: [line.split(" ")]).filter(lambda x: len(x) == 2)\
        .map(lambda line: (line[0],line[1]))\
        .reduceByKey(lambda x,y:x if len(x) >= len(y) else y)\
        .updateStateByKey(updateFunc)
pairs.foreachRDD(printre)

good = pairs.filter(lambda x: x[0] == 'good')
good.foreachRDD(printgood)


ssc.start()
ssc.awaitTermination(50)
ssc.stop(False)
print("Finished")

[('social', 'constructionism'), ('coercive', 'deprogramming'), ('individual', 'meaninglessness'), ('political', 'self-determination'), ('desirable', 'characteristic')]
GOOD: ['morning/afternoon']

[('social', 'constructionism'), ('coercive', 'deprogramming'), ('individual', 'meaninglessness'), ('political', 'self-determination'), ('desirable', 'characteristic')]
GOOD: ['morning/afternoon']

[('social', 'constructionism'), ('coercive', 'administration'), ('individual', 'meaninglessness'), ('political', 'assassinations-were'), ('desirable', 'characteristic')]
GOOD: ['morning/afternoon']

[('social', 'constructionism'), ('coercive', 'administration'), ('individual', 'being-in-the-world'), ('political', 'assassinations-were'), ('desirable', 'characteristic')]
GOOD: ['morning/afternoon']

[('social', 'realism/constructivism'), ('coercive', 'administration'), ('individual', 'being-in-the-world'), ('political', 'assassinations-were'), ('desirable', 'characteristic')]
GOOD: ['morning/afternoon

In [16]:
p = 4
rdd = sc.parallelize([1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,1,1,1,1,1,1], p)
rdd.collect()

[1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1]

In [23]:
def divide(it):
    left_one = 0    # Number of continuous 1's counted from left
    right_one = 0    # Number of continuous 1's counted from right
    local_one = 0    # The longest continuous 1's within partition
    indicator = True # indicates whether the partition is all 1's
    running_one = 0
    
    for x in it:
        if (x == 0):
            if (indicator): # Encountered the first zero
                left_one = running_one 
            if (running_one > local_one):
                local_one = running_one
            running_one = 0
            indicator = False
        else:
            running_one += 1
    if local_one < running_one:
        local_one = running_one
    right_one = running_one
    if (indicator): # update the case of all 1's
        left_one = running_one
        local_one = running_one
            
    yield [left_one, right_one, local_one, indicator]
    
    
L = rdd.mapPartitions(divide).collect()
print(L)
def conquer(L):
    res = 0
    running_one = 0
    
    for x in L:
        (left_one, right_one, local_one, indicator) = x
        # case 1, maximum appears within site
        if (local_one > res):
            res = local_one
        # case 2 and 3, maximum cross multiple partitions
        if (indicator): # carry to the next partition
            running_one += local_one
        else: # solve the cross-partition max
            running_one += left_one
            if (running_one > res):
                res = running_one
            running_one = right_one
    # deal with the end
#     if (running_one > res):
#         res = running_one
    return res
            
print(conquer(L))


[[1, 1, 1, False], [3, 0, 3, False], [3, 0, 3, False], [0, 6, 6, False]]
6


In [20]:
print(rdd.glom().collect())

[[1, 0, 0, 1], [1, 1, 1, 0], [1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1]]


Past Exam Examples
Past Exam Examples
Past Exam Examples
Past Exam Examples
Past Exam Examples
Past Exam Examples
Past Exam Examples
Past Exam Examples
Past Exam Examples
Past Exam Examples
Past Exam Examples

In [56]:
#2020 part2 example2
RDD1 = sc.parallelize(range(1, 10))
RDD2 = sc.parallelize(range(1, 10))
r1 = RDD1.zipWithIndex().map(lambda x: (x[1],x[0]))
print(r1.collect())
r2 = RDD2.zipWithIndex().map(lambda x: (x[1],x[0]))
RDD3 = r1.union(r2).reduceByKey(lambda x,y: x*y).map(lambda x: x[1])
dot_product = RDD3.reduce(lambda x,y: x+y)
print(dot_product)

[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9)]
285


In [1]:
#2020 part2 example 3
from pyspark.streaming import StreamingContext

ssc = StreamingContext(sc, 5)
# Provide a checkpointing directory.  Required for stateful transformations
ssc.checkpoint("checkpoint")

rdd = sc.textFile('adj_noun_pairs.txt', 8).map(lambda l: tuple(l.split())).filter(lambda p: len(p)==2)
rddQueue = rdd.randomSplit([1]*10, 123)
lines = ssc.queueStream(rddQueue)

def updateFunc(newValues, runningCount):
    if runningCount is None:
        runningCount = 0
    return sum(newValues, runningCount)

running_counts = lines.map(lambda x: (x,1))\
                      .updateStateByKey(updateFunc)

counts_sorted = running_counts.transform(lambda rdd: rdd.sortBy(lambda x: x[1], False))
                            #in stream need to use transform to do rdd functions

# counts_sorted = running_counts.map(lambda x: (x[0][1], x)). \
#                                reduceByKey(lambda x,y: x if x[1] > y[1] else y). \
#                                map(lambda x: x[1]). \
#                                transform(lambda rdd: rdd.sortBy(lambda x: x[1], False))

def printResults(rdd):
    print(rdd.take(10))

counts_sorted.foreachRDD(printResults)


ssc.start()
ssc.awaitTermination(5)
ssc.stop(False)
print("Finished")

[(('external', 'link'), 836), (('19th', 'century'), 327), (('same', 'time'), 280), (('20th', 'century'), 266), (('first', 'time'), 264), (('other', 'hand'), 236), (('large', 'number'), 227), (('civil', 'war'), 211), (('political', 'party'), 201), (('recent', 'year'), 189)]
Finished


In [3]:
sum([],5)

5

In [10]:
#2019 F Q4
rdd1 = sc.parallelize([5,0,0,3])
rdd2 = sc.parallelize([3,4,6,2])

def ele_wise_add(rdd1, rdd2): 
    rdd1 = rdd1.zipWithIndex().map(lambda x:(x[1],x[0]))
    rdd2 = rdd2.zipWithIndex().map(lambda x:(x[1],x[0]))
    return rdd1.union(rdd2).reduceByKey(lambda x,y:x+y).map(lambda x:x[1])

rdd3 = ele_wise_add(rdd1, rdd2)
print(rdd3.collect())
# Expected output: [8,4,6,5]

[8, 4, 6, 5]


In [32]:
e=sc.parallelize([(1,2),(1,3),(1,4),(2,3),(3,1)]).toDF(["src","dst"])

e1=e.withColumnRenamed('src', 'x').withColumnRenamed('dst','y')
e2=e.withColumnRenamed('src', 'y1').withColumnRenamed('dst','z')
e3=e.withColumnRenamed('src', 'z1').withColumnRenamed('dst','x1')
result = e1.join(e2).join(e3).where("y = y1 AND z = z1 AND x = x1 AND x < y AND y < z")
result.select('x','y','z').show()

+---+---+---+
|  x|  y|  z|
+---+---+---+
|  1|  2|  3|
+---+---+---+



In [37]:
dfCustomer = spark.read.csv('customer1.csv', header=True, inferSchema=True)
dfOrders = spark.read.csv('orders.csv', header=True, inferSchema=True)
from pyspark.sql.functions import *
dfOrdersGroup = dfOrders.groupBy("CUSTKEY").count()
dfOrdersGroup.createOrReplaceTempView("orders")
dfCustomer.createOrReplaceTempView("customer")
result = spark.sql("SELECT customer.CUSTKEY, count FROM customer left outer join orders on customer.CUSTKEY = orders.CUSTKEY")\
        .na.fill(0)\
        .withColumnRenamed("count", "order_count").groupBy("order_count").count()\
        .withColumnRenamed("count","cust_count")\
        .sort(desc("cust_count"),desc("order_count"))
result.show()

+-----------+----------+
|order_count|cust_count|
+-----------+----------+
|          0|       500|
|         11|        67|
|         12|        63|
|         10|        63|
|          9|        63|
|          8|        62|
|         14|        57|
|         20|        55|
|         13|        50|
|         15|        45|
|         21|        44|
|          7|        43|
|         18|        42|
|         16|        42|
|         17|        40|
|         24|        36|
|         22|        36|
|         19|        36|
|          6|        32|
|         23|        25|
+-----------+----------+
only showing top 20 rows



In [50]:
acc = sc.accumulator(0)
def f(x):
     acc.add(x)
# sc.parallelize([1,2,3,4]).foreach(lambda x:f(x))
print(sc.parallelize([1,2,3,4]).map(lambda x:f(x)).collect())
print(acc)

[None, None, None, None]
10


In [53]:
#2019 Q2
numPartitions = 10

points = sc.textFile('points.txt',numPartitions)
pairs = points.map(lambda l: tuple(l.split()))
pairs = pairs.map(lambda pair: (int(pair[0]),int(pair[1])))
pairs.cache()

def f(iterator):
    dump = list(iterator)
    for P in dump:
        k = 1
        for Q in dump:
            if P[0] == Q[0] and P[1] == Q[1]:
                continue
            if P[0] <= Q[0] and P[1] <= Q[1]:
                k = 0
                break
        if k == 1:
            yield P

temp = pairs.mapPartitions(f).collect()
result = []
for i in temp:
    k = 1;
    for j in temp:
        if i[0] == j[0] and i[1] == j[1]:
            continue
        if i[1] <= j[1] and i[0] <= j[0] :
            k = 0
            break
    if k == 1 :
        result.append(i)


pairs.mapPartitions(f).collect()

# print result

[(99747, 35557),
 (93036, 99441),
 (98158, 92386),
 (99623, 73826),
 (99978, 19775),
 (99210, 88620),
 (41277, 99990),
 (94106, 99309),
 (99773, 34927),
 (95716, 98344),
 (99989, 21634),
 (99480, 95038),
 (98610, 95551),
 (99958, 59581),
 (95086, 99479),
 (99610, 63697),
 (92580, 99974),
 (96525, 96473),
 (99202, 47009),
 (92327, 99855),
 (95598, 93566),
 (99476, 46250),
 (98590, 88094),
 (98766, 75551),
 (94187, 99665),
 (99799, 8240),
 (95312, 95227),
 (97493, 88501),
 (26419, 99969),
 (8928, 99976),
 (80530, 99587),
 (98814, 98537),
 (99958, 26054),
 (99956, 79605),
 (99740, 98217),
 (88007, 98944),
 (41391, 99972),
 (78056, 99970),
 (59920, 99992),
 (98231, 94688),
 (99568, 91339),
 (96815, 99398),
 (99913, 68052),
 (96167, 99795),
 (51623, 99893),
 (95316, 98802),
 (98192, 94109),
 (96629, 98467),
 (99969, 11482),
 (90667, 99089),
 (94683, 99039),
 (79046, 99675),
 (59329, 99737),
 (99010, 77446),
 (99662, 75927),
 (98161, 84105),
 (99652, 83201),
 (87366, 98476),
 (45347, 99791),

In [55]:
#2018 Q1
dfCustomer = spark.read.csv('Customer.csv', header=True, inferSchema=True)
dfDetail = spark.read.csv('SalesOrderDetail.csv', header=True, inferSchema=True)
dfHeader = spark.read.csv('SalesOrderHeader.csv', header=True, inferSchema=True)
dfCustomer.show()
##Not Finish

+----------+---------+-----+-----------+----------+----------+------+--------------------+--------------------+--------------------+-------------------+--------------------+------------+--------------------+--------------------+
|CustomerID|NameStyle|Title|  FirstName|MiddleName|  LastName|Suffix|         CompanyName|         SalesPerson|        EmailAddress|              Phone|        PasswordHash|PasswordSalt|             rowguid|        ModifiedDate|
+----------+---------+-----+-----------+----------+----------+------+--------------------+--------------------+--------------------+-------------------+--------------------+------------+--------------------+--------------------+
|         1|        0|  Mr.|    Orlando|        N.|       Gee|  NULL|        A Bike Store|adventure-works\p...|orlando0@adventur...|       245-555-0173|L/Rlwxzp4w7RWmEgX...|    1KjXYs4=|3F5AE95E-B87D-4AE...|2005-08-01 00:00:...|
|         2|        0|  Mr.|      Keith|      NULL|    Harris|  NULL|  Progressive S

In [73]:
#2018 Q3
numPartitions = 10
A = (0,0)
B = (10000,0)
C = (0,10000)

points = sc.textFile('points.txt',numPartitions)
pairs = points.map(lambda l: tuple(l.split()))
pairs = pairs.map(lambda pair: (int(pair[0]),int(pair[1])))
pairs.cache()

def f(iterator):
    counter = 0
    for i in iterator:
        if i[0] >= 0 and i[0] <=10000 and i[1] >= 0 and i[1] <= - i[0] + 10000:
            counter += 1
    yield counter

temp = pairs.mapPartitions(f).collect()

print(temp)
# result = sum(temp)

# print(result)


[3, 4, 5, 2, 7, 1, 5, 2, 7, 5]


In [20]:
array = [1, 2, 1, 1, 2, 2, 1, 1]
s = 4

from pyspark import SparkContext
RDD = sc.parallelize(array, 2).cache()

def sequential(nums):
    i = 0
    j = 0
    counter = 0
    global s
    while(j<=len(nums)):
        if sum(nums[i:j]) == s:
            counter += 1
            i += 1
        elif sum(nums[i:j]) < s:
            j += 1
        else:
            i += 1
    return counter

def divide(it):
    L = []
    R = []
    i = 0
    j = 0
    counter = 0
    li = []
    for i in it:
        li.append(i)
    global s
    while(j<=len(li)):
        if sum(li[i:j]) == s:
            if i == 0:
                L = li[i:j-1]
            counter += 1
            i += 1
        elif sum(li[i:j]) < s:
            if j == len(li):
                R = li[i:j]
            j += 1
        else:
            if i == 0:
                L = li[i:j-1]
            i += 1
    return [[L,R,counter]]

print(divide([1,1,1,1,1,1,1]))
        

def conquer(X):
    counter = 0
    for i in range(len(X)):
        if i != 0 and i != len(X)-1:
            counter += sequential(X[i][1] + X[i+1][0])
        counter += X[i][2]
    return counter
        

# X = RDD.mapPartitions(divide).collect()
# print(X)
# result = conquer(X)
# print(result)

# END OF YOUR CODE

[[[], [], 0]]
